EKF for tracking with time-of-arrival (ToA) estimates.
 
INPUTS
 r_ges:    measured distances (M x N)
 theta_init:  initial state estimate
 BS:   base station positions, M x 2 np array
OUTPUTS
 th_hat:             state estimates
 P_min:              apriori covariance
 P:                  aposteriori covariance

# TODO

* P_min false values
* P false values

In [ ]:
r_ges =

    0.5377   -1.3499    0.6715    0.8884   -0.1022
    1.8339    3.0349   -1.2075   -1.1471   -0.2414
   -2.2588    0.7254    0.7172   -1.0689    0.3192
    0.8622   -0.0631    1.6302   -0.8095    0.3129
    0.3188    0.7147    0.4889   -2.9443   -0.8649
   -1.3077   -0.2050    1.0347    1.4384   -0.0301
   -0.4336   -0.1241    0.7269    0.3252   -0.1649
    0.3426    1.4897   -0.3034   -0.7549    0.6277
    3.5784    1.4090    0.2939    1.3703    1.0933
    2.7694    1.4172   -0.7873   -1.7115    1.1093

In [30]:
import numpy as np
import scipy.io

def ekf_toa(r_ges, theta_init, BS, parameter=None):
    if parameter is None:
        print("parameters are set to default")
        parameter = {}
        sigma_v = 1;
        M = BS.shape[0] # M numer of BS, N number of samples
        P0= np.diag([100, 100, 10,10]) # initial state covariance
        R = 150**2 * np.diag(np.ones(M)) # measurement covariance
        Ts= 0.2 # sampling frequency
        A = np.array([[1, 0, Ts, 0],[ 0, 1, 0, Ts],[ 0, 0, 1, 0],[ 0, 0, 0, 1]]) # state transition matrix
        Q = sigma_v**2 * np.eye(2)
        G = np.vstack((0.5*Ts**2*np.eye(2), Ts*np.eye(2)))
    else:
        P0 = parameter['P0']
        R  = parameter['R']
        Q  = parameter['Q']
        G  = parameter['G']
        A  = parameter['A']
    ''' 
    if 2*parameter['dim'] != len(theta_init[:,0]) or 2*parameter['dim'] != len(P0[:,0]):
        disp('State vector or state covariance do not match the dimensions of the BS')
    '''    
    x = BS[:,0]
    y = BS[:,1]
    
    M = len(x)
    N = len(r_ges[0,:])
    
    P = np.zeros((4,4,N))
    P[:,:,0] = P0
    th_hat = np.zeros((4,N))
    th_hat[:,0] = theta_init.flatten()
    th_hat_min = np.zeros([4,N])
    P_min = np.zeros([4,4,N])
    H = np.zeros((M,4))
    # H[M,4] = 0
    h_min = np.zeros(M)
    sigma2 = np.zeros(N)
    
    for kk in range(1,N):
        th_hat_min[:,kk] = A@th_hat[:,kk-1]
        
        for ii in range(M):
            H[ii,:] = [(th_hat_min[0,kk]-x[ii])/\
                       np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                               + (th_hat_min[1,kk]-y[ii])**2) ,\
                       (th_hat_min[1,kk]-y[ii])/\
                       np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                               + (th_hat_min[1,kk]-y[ii])**2)\
                       ,0,0]
            h_min[ii] = np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                               + (th_hat_min[1,kk]-y[ii])**2)
            
        '''
        if parameter['var_est'] == 1:
            sigma = 1.483*np.mean(np.abs((r_ges[:,kk] -h_min.T)-\
                                        np.median(r_ges[:,kk] -h_min.T)))
            sigma2[kk] = sigma**2
            R = sigma2[kk]*np.eye(M)
        '''
        
        P_min[:,:,kk] = A@P[:,:,kk-1]@A.T + G@Q@G.T
        K = P_min[:,:,kk]@H.T@np.linalg.inv(H@P_min[:,:,kk]@H.T+R)
        
        th_hat[:,kk] = th_hat_min[:,kk] + K@(r_ges[:,kk]- h_min.T)
        P[:,:,kk] = (np.eye(4) - K@H)@P_min[:,:,kk]
        
    parameter['Rest'] = sigma2
    parameter['K'] = K
    return th_hat, P_min, P, parameter

r_ges = scipy.io.loadmat('r_ges.mat',struct_as_record=False)['r_ges']
BS = scipy.io.loadmat('BS.mat',struct_as_record=False)['BS']
theta_init = scipy.io.loadmat('theta_init.mat',struct_as_record=False)['theta_init']

th_hat, P_min, P, parameter = ekf_toa(r_ges, theta_init, BS )
P

parameters are set to default


array([[[ 1.00000000e+02,  9.84071432e+01,  9.76618736e+01,
          9.77154436e+01,  9.85170386e+01],
        [ 0.00000000e+00, -1.56204283e+00, -2.99616815e+00,
         -4.33865203e+00, -5.61182531e+00],
        [ 0.00000000e+00,  1.96421443e+00,  3.89842696e+00,
          5.80298430e+00,  7.67658642e+00],
        [ 0.00000000e+00, -3.11784997e-02, -8.87796038e-02,
         -1.69043169e-01, -2.68433819e-01]],

       [[ 0.00000000e+00, -1.56204283e+00, -2.99616815e+00,
         -4.33865203e+00, -5.61182531e+00],
        [ 1.00000000e+02,  9.80616095e+01,  9.70195287e+01,
          9.68082776e+01,  9.73724463e+01],
        [ 0.00000000e+00, -3.11784997e-02, -8.87731074e-02,
         -1.69012912e-01, -2.68345918e-01],
        [ 0.00000000e+00,  1.95731755e+00,  3.87960536e+00,
          5.76831948e+00,  7.62337237e+00]],

       [[ 0.00000000e+00,  1.96421443e+00,  3.89842696e+00,
          5.80298430e+00,  7.67658642e+00],
        [ 0.00000000e+00, -3.11784997e-02, -8.87731074e-02,


In [10]:
import numpy as np

np.diag([100, 100, 10,10])
np.eye(2)
len(np.ones(8))
np.zeros((4,4,6))
a = np.zeros([4,7])
a[1,:] = [1,2,3\
          ,4,5,\
          6,7]
a

array([[0., 0., 0., 0., 0., 0., 0.],
       [1., 2., 3., 4., 5., 6., 7.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])